# Titanic: Machine Learning from Disaster

## Una solución usando árboles aleatorios

En esta libreta se explicará por partes una solución para el problema del Titanic usando árboles de decisión.
El código original puede ser consultado en la siguiente página:

https://www.kaggle.com/kushal1412/titanic/decision-tree-survivors/code

El autor del código no lo explica a detalle pero es fácil de comprender con la información que se tiene del problema y usando como referencia la documentación siguiente de sklearn:

http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html



Comenzamos importando las librerias necesarias

In [1]:
import numpy as np
import pandas as pd
from sklearn import cross_validation as cv
from sklearn.cross_validation import KFold
from sklearn.tree import DecisionTreeClassifier

from sklearn.neighbors import KNeighborsClassifier                         # KNN
from sklearn.svm import SVC                                                # SVM
from sklearn.ensemble import RandomForestClassifier                        # Bósque aleatorios
from sklearn.ensemble import AdaBoostClassifier                            # ADA Boost
from sklearn.naive_bayes import GaussianNB                                 # Naive bayes
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis       # Logística sin regularización
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis    # Logística con polinomio de orden 2
from sklearn.linear_model import  LogisticRegression                       # Logística con regularización

Luego sigue importar los datos de entrenamiento y los de prueba.

In [2]:
print('Consiguiendo los datos de entrenamiento y el conjunto de prueba...')
train = pd.read_csv("train.csv", dtype={"Age": np.float64})
test  = pd.read_csv("test.csv", dtype={"Age": np.float64})

print 'Datos de entrenamiento:', len(train)

print 'Datos de prueba:', len(test)

Consiguiendo los datos de entrenamiento y el conjunto de prueba...
Datos de entrenamiento: 891
Datos de prueba: 418


Tendremos una función que "armoniza" los datos. Su trabajo es llenar campos vacíos y asignar un valor numérico a los datos que no lo sean (sexo y lugar de embarcación)

In [3]:
def harmonize_data(titanic):
    # Llena campos vacíos
    titanic["Age"] = titanic["Age"].fillna(titanic["Age"].mean())
    titanic["Fare"] = titanic["Fare"].fillna(titanic["Fare"].mean())
    titanic["Embarked"] = titanic["Embarked"].fillna("S")
    # Asigna valores numéricos a los datos para facilitar cálculos
    titanic.loc[titanic["Sex"] == "male", "Sex"] = 1
    titanic.loc[titanic["Sex"] == "female", "Sex"] = 0
    titanic.loc[titanic["Embarked"] == "S", "Embarked"] = 0
    titanic.loc[titanic["Embarked"] == "C", "Embarked"] = 1
    titanic.loc[titanic["Embarked"] == "Q", "Embarked"] = 2
    return titanic

Con esto evitaremos errores que pudiera causar un campo vacio.

Usamos la función en todos los datos que usaremos para el entrenamiento y el proceso de prueba.

In [4]:
train_data = harmonize_data(train)
test_data  = harmonize_data(test)

De una vez declaramos la función que usaremos más adelante para guardar nuestrar predicciones en un archivo de Excel. En la primera columna se muestra el ID de cada pasajero y a su derecha la predicción de si sobrevivirán o no.

In [5]:
def create_submission(dtc, train, test, predictors, filename):
    dtc.fit(train[predictors], train["Survived"])
    predictions = dtc.predict(test[predictors])
    submission = pd.DataFrame({
        "PassengerId": test["PassengerId"],
        "Survived": predictions
    })
    submission.to_csv(filename, index=False)

Con nuestros datos ya procesados, agregaremos un par de campos.
* El campo PSA es el producto del valor de clase, sexo y edad del pasajero. Recordemos que a los campos de clase y sexo se les asignó un valor numérico.
* El campo SP contiene el número de familiares del pasajero a bordo, dentro de los cuales se toman en cuenta hijos, esposos, hermanos y padres.

In [6]:
train_data["PSA"] = train_data["Pclass"]*train_data["Sex"]*train_data["Age"]
train_data["SP"] = train_data["SibSp"]+train_data["Parch"]
test_data["PSA"] = test_data["Pclass"]*test_data["Sex"]*test_data["Age"]
test_data["SP"] = test_data["SibSp"]+test_data["Parch"]

Ahora fijaremos nuestros "predictores", que serán los atributos que tomaremos en cuenta de cada pasajero para calcular predicciones. Estos campos son:
* Clase
* Sexo
* Edad
* PSA
* Cantidad que pagó por el boleto
* Lugar de embarcación
* SP

In [7]:
predictors = ["Pclass", "Sex", "Age", "PSA", "Fare", "Embarked", "SP"]

En *sklearn* los árboles incluyen la función *score*, la cual regresa la precisión de los datos de prueba, dando como parámetros estos datos y las clases a las que pertenece cada uno. Usaremos esto para calcular la profundidad (llamada *max_depth* por el algoritmo) que nos dé los resultados más acertados, probando cada caso del 1 al 100.

In [8]:
max_score = 0
best_n = 0
for n in range(1,100):
    dtc_scr = 0.
    dtc = DecisionTreeClassifier(max_depth=n)
    for train, test in KFold(len(train_data), n_folds=10, shuffle=True):
        dtc.fit(train_data[predictors].T[train].T, train_data["Survived"].T[train].T)
        dtc_scr += dtc.score(train_data[predictors].T[test].T, train_data["Survived"].T[test].T)/10
    if dtc_scr > max_score:
        max_score = dtc_scr
        best_n = n
print(best_n, max_score)

(4, 0.8205118601747815)


Otro parámetro del arbol de decisión es *min_samples_split*, el cual es el mínimo número de muestras necesarias para partir un nodo. Al igual que con la profundidad, buscaremos el mejor valor para nuestro algoritmo.

In [9]:
max_score = 0
best_s = 0
for s in range(1,100):
    dtc_scr = 0.
    dtc = DecisionTreeClassifier(min_samples_split=s)
    for train, test in KFold(len(train_data), n_folds=10, shuffle=True):
        dtc.fit(train_data[predictors].T[train].T, train_data["Survived"].T[train].T)
        dtc_scr += dtc.score(train_data[predictors].T[test].T, train_data["Survived"].T[test].T)/10
    if dtc_scr > max_score:
        max_score = dtc_scr
        best_s = s
print(best_s, max_score)

(23, 0.83389513108614222)


Luego de esto ya podemos crear nuestro arbol de decisión. El código original usaba como criterio *entropy* y un separador de nodos *random*. El criterio puede cambiarse a *gini* para usar la impureza de Gini en vez de ganancia de información, y como estrategia para el separador se puede cambiar a *best* para elegir la mejor separación (*random* usa la mejor separación aleatoria).

In [10]:
print('Haciendo predicciones...')
dtc = DecisionTreeClassifier(max_depth=best_n, min_samples_split=best_s, criterion='entropy', splitter='random')
print('Creando archivo Excel...')
create_submission(dtc, train_data, test_data, predictors, "dtcsurvivors.csv")
print('Listo.')

Haciendo predicciones...
Creando archivo Excel...
Listo.


Vamos a hacer unas modificaciones al algoritmo para ver cómo cambian los resultados. Cambiaremos el criterio al *default* y el separador a *best*; o lo que es lo mismo: no asignarles valor y usarán estos por default.

In [11]:
print('Haciendo predicciones...')
dtc = DecisionTreeClassifier(max_depth=best_n, min_samples_split=best_s)
print('Creando archivo Excel...')
create_submission(dtc, train_data, test_data, predictors, "dtcsurvivors2.csv")
print('Listo.')

Haciendo predicciones...
Creando archivo Excel...
Listo.
